# Qiskit pulseを使って超伝導量子ビットを操作する

In [ ]:
import numpy as np
import scipy.optimize as sciopt
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, pulse, transpile
from qiskit.circuit import Gate, Parameter
from qiskit_experiments.test import SingleTransmonTestBackend
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_provider import least_busy
%matplotlib inline

## 回路を実行して"1"が得られる確率を返す関数

In [ ]:
def execute(backend, physical_qubit, circuits, shots=10000, session=None, tag=''):

    circuits = transpile(circuits, backend=backend, initial_layout=[physical_qubit], optimization_level=0)
    if session:
        inputs = {'circuits': circuits, 'shots': shots}
        if tag:
            options = {'job_tags': [tag]}
        counts = session.run('circuit-runner', inputs, options=options).result().get_counts()
    else:
        run_options = {'shots': shots}
        if tag:
            run_options['job_tags'] = [tag]
        counts = backend.run(circuits, **run_options).result().get_counts()

    yval = np.array([c.get('1', 0) / shots for c in counts])
    
    return yval

## カーブフィットをする関数

In [ ]:
def fit(curve, xval, yval, p0, shots):
    popt, _ = sciopt.curve_fit(curve, xval, yval, p0=p0)

    plt.errorbar(xval, yval, yerr=np.sqrt(yval * (1. - yval) / shots), fmt='o', markersize=3)
    xfine = np.linspace(xval[0], xval[-1], 400)
    plt.plot(xfine, curve(xfine, *popt))

    return popt

## まずはシミュレーターを較正する

In [ ]:
backend = SingleTransmonTestBackend(noise=False)
physical_qubit = 0

In [ ]:
def rough_frequency_calibration(backend, physical_qubit, shots=10000, session=None):
   
    frequency = Parameter('frequency')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='spectrocopy') as spectroscopy_sched:
        pulse.shift_frequency(frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=512, amp=0.01, sigma=128), drive_channel)

    frequencies = np.linspace(-1.e+8, 1.e+8, 41)
    
    circuits = []
    for freq_value in frequencies:
        sched = spectroscopy_sched.assign_parameters({frequency: freq_value}, inplace=False)
        
        circuit = QuantumCircuit(1)
        circuit.append(Gate('spectroscopy', 1, []), [0])
        circuit.measure_active()
    
        circuit.add_calibration('spectroscopy', [physical_qubit], sched)

        circuits.append(circuit)

    yval = execute(backend, physical_qubit, circuits, shots=shots, session=session)

    def resonance_curve(x, f0, a, b, sigma):
        return a * np.exp(-(x - f0)**2 / 2. / sigma**2) + b

    ipeak = len(frequencies) // 2
    f0_guess = frequencies[ipeak]
    a_guess = yval[ipeak] - yval[0]
    b_guess = yval[0]
    half_max = np.nonzero(yval[ipeak:] - (yval[ipeak] / 2) < 0)[0][0] + ipeak
    sigma_guess = frequencies[half_max] - frequencies[ipeak]

    p0 = (f0_guess, a_guess, b_guess, sigma_guess)

    popt = fit(resonance_curve, frequencies, yval, p0, shots)

    qubit_frequency = popt[0] + backend.target.qubit_properties[physical_qubit].frequency

    return qubit_frequency

In [ ]:
qubit_frequency = rough_frequency_calibration(backend, physical_qubit)

In [ ]:
def rough_amplitude_calibration(backend, physical_qubit, qubit_frequency, shots=10000, session=None):

    amp = Parameter('amp')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='rabi') as rabi_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=amp, sigma=40), drive_channel)
    
    amplitudes = np.linspace(0., 0.2, 40)
    
    circuits = []
    for amp_val in amplitudes:
        circuit = QuantumCircuit(1)
        circuit.append(Gate('rabi', 1, []), [0])
        circuit.measure_active()
        
        circuit.add_calibration('rabi', [physical_qubit],
                                rabi_sched.assign_parameters({amp: amp_val}, inplace=False))
    
        circuits.append(circuit)

    yval = execute(backend, physical_qubit, circuits, shots=shots, session=session)
    
    def oscillation_curve(x, freq, phase, a, b):
        return a * np.cos(x * freq + phase) + b

    indices = np.nonzero(yval > 0.9)[0]
    if len(indices):
        first_above_09 = indices[0]
        indices = np.nonzero(yval[first_above_09:] < 0.1)[0]
        if len(indices):
            next_below_01 = [0] + first_above_09
            first_max = np.argmax(yval[:next_below_01])
            freq_guess = np.pi / amplitudes[first_max]
        else:
            freq_guess = np.pi / 2. / amplitudes[first_above_09]
    else:
        freq_guess = 0.1 * np.pi / amplitudes[-1]
    phase_guess = 0.
    a_guess = -0.5
    b_guess = 0.5

    p0 = (freq_guess, phase_guess, a_guess, b_guess)

    popt = fit(oscillation_curve, amplitudes, yval, p0, shots)

    x_amp = np.pi / popt[0]
    sx_amp = x_amp / 2.

    return x_amp, sx_amp

In [ ]:
x_amp, sx_amp = rough_amplitude_calibration(backend, physical_qubit, qubit_frequency)

In [ ]:
def fine_frequency_calibration(backend, physical_qubit, qubit_frequency, sx_amp, shots=10000, session=None):

    delay_unit = 32
    ndelay = Parameter('ndelay')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='ramsey') as ramsey_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=sx_amp, sigma=40), drive_channel)
        pulse.delay(ndelay * delay_unit, drive_channel)
        pulse.shift_phase(np.pi / 2. * ndelay, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=sx_amp, sigma=40), drive_channel)
            
    ndelays = np.arange(32)
    
    circuits = []
    for ndelay_val in ndelays:
        circuit = QuantumCircuit(1)
        circuit.append(Gate('ramsey', 1, []), [0])
        circuit.measure_active()
        
        circuit.add_calibration('ramsey', [physical_qubit],
                                ramsey_sched.assign_parameters({ndelay: ndelay_val}, inplace=False))
    
        circuits.append(circuit)

    yval = execute(backend, physical_qubit, circuits, shots=shots, session=session)
    
    def error_amplification_curve(x, delta, a, b, offset):
        return a * np.cos(0.5 * np.pi * x + delta * (x + 1) + offset) + b
    
    delta_guess = 0.
    a_guess = 0.5
    b_guess = 0.5
    offset_guess = 0.

    p0 = (delta_guess, a_guess, b_guess, offset_guess)

    popt = fit(error_amplification_curve, ndelays, yval, p0, shots)

    refined_qubit_frequency = qubit_frequency - popt[0] / (delay_unit * backend.dt * 2. * np.pi)

    return refined_qubit_frequency

In [ ]:
refined_qubit_frequency = fine_frequency_calibration(backend, physical_qubit, qubit_frequency, sx_amp)

### 周波数は実際の値に近くなったか？

In [ ]:
print(backend.target.qubit_properties[physical_qubit].frequency - refined_qubit_frequency)
print(backend.target.qubit_properties[physical_qubit].frequency - qubit_frequency)

## 実機を利用

In [ ]:
runtime_service = QiskitRuntimeService(channel='ibm_quantum', instance='ibm-q-utokyo/internal/qc-training2023s')

In [ ]:
backend_list = runtime_service.backends(filters=lambda b: 'simulator' not in b.name and b.name != 'ibm_oslo' and b.status().operational)
# 今一番空いているバックエンド
backend = least_busy(backend_list)
print(backend.name)

In [ ]:
with Session(service=runtime_service, backend=backend) as session:
    qubit_frequency = rough_frequency_calibration(backend, physical_qubit, session=session)
    x_amp, sx_amp = rough_amplitude_calibration(backend, physical_qubit, qubit_frequency, session=session)
    refined_qubit_frequency = fine_frequency_calibration(backend, physical_qubit, qubit_frequency, sx_amp, session=session)